# Lecture 3.2: Attack on AES with Differential-Power-Analysis (Kocher et al. 1999) using Lascar

#### Learning goals
- Learn how to apply a DPA attack using Lascar

#### References
- https://github.com/Ledger-Donjon/lascar

In [2]:
%load_ext autoreload
%autoreload 2

import os
import random

import numpy as np
import plotly.graph_objects as pgo
from cwtoolbox import CaptureDevice

## Why Lascar?

When it comes to "real" side-channel analysis the author favours [Lascar](https://github.com/Ledger-Donjon/lascar) with the following reasons:

- High performance
- Well structured and documented code
- Easy start due to good examples
- Citations given for the methods of analysis

## Main aspects of lascar

While lascar's tutorial and the examples provide a comprehensive entry point the main concepts are demonstrated here to make the subsequent analysis understandable.

In [3]:
import lascar

### Container

In [ ]:
import sys
print(sys.modules['lascar.container'].__doc__)

We will mainly use the following container:

In [ ]:
lascar.TraceBatchContainer

As described [here](https://github.com/Ledger-Donjon/lascar/blob/master/tutorial/01-discovering-containers.py) it takes to items:


A side-channel trace (Trace in lascar), is a couple of two items:
- The first item is "leakage" and represents the side-channel observable
- The second item is "value" and represents the handled values during the observation of "leakage".
The only restriction here is that "leakage" and "data" must be numpy.arrays of any shape.
trace = (leakage, value) where leakage and value are numpy.arrays
The __str__ method of Trace displays the shape and dtype for both leakage and value.

### Engine

The most important concept is `Engine`. It performs the actual analysis.

In [ ]:
print(lascar.Engine.__doc__)

One example is `DpaEngine`.

In [ ]:
print(lascar.DpaEngine.__doc__)

### OutputMethod

The data which is produced by an Engine is passed to an `OutputMethod.`

In [23]:
print(lascar.OutputMethod.__doc__)


    OutputMethod is an virtual class.
    Its role is to set up the strategy for outputing results computed by engines.

    The Session registers an OutputMethod.
    The OuputMethod tracks engines, and, at each output_step, it request results to them.

    The children inherating from OutputManager implements how to process these results, and how to deliver them to the user.
    


### Session

The final ingredient to perform an analysis is a `Session`.

In [22]:
print(lascar.Session.__doc__)


    This class is leading side-channel operation in lascar.

    a :class:`lascar.session.Session` object's role is to:

    - get batch of side channel traces from a Container, 'container'
    - distribute the batchs to the registered engines. 'engines'
    - manage outputs thanks to 'output_method', 'output_step'
        
    :param container: the container that will be read during the session. Only
        mandatory argument for constructor.
    :param engine: lascar engine to be registered by the Session
    :param engines: list of lascar engines to be registered by the Session
    :param output_method: specify the output method: how will the results from
        the engine will be manipulated. see lascar/output for more info.
    :param output_steps: specify when the Session will ask its engines to
        compute results.
    :param name: name given for the Session.
    :param progressbar: Will the Session display a progressbar during its
        process.
    


### Note

Lascar uses `numba.jit` to speed up calculations.
This "costs" an initial effort when compiling the functions but provides awesome speed afterwards.
Jit is enabled by default and can be configured for some engines.

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 1</div>

Perform a DPA attack on [sbox_lookup.c](sbox_lookup.c) on the LSB of the output of the first SBox lookup using Lascar. Use
- `TraceBatchContainer`
- `DpaEngine`
- `ConsoleOutputMethod`

</div>

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 2 (optional)</div>

Perform the attack using two output modes together: `ConsoleOutputMode` and `MatplotlibOutputMethod`.

</div>

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 3 (optional)</div>

Perform a full DPA attack (i.e. reveal all key bytes) using Lascar.

Hints:
- Multiple Engines are needed. One for each selection function.
- 16 different selection functions are needed. Create a function that returns a selection function.

</div>

In [4]:
capture_device = CaptureDevice.create("CWLITEXMEGA")
capture_device.compile(file=os.path.abspath("sbox_lookup.c"))
capture_device.flash()

data = capture_device.capture(
    number_of_traces=1000,
    input=lambda _: list(random.randbytes(16)),
)

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2513 bytes


100%|██████████| 1000/1000 [00:08<00:00, 116.11it/s]


In [8]:
data["input"]

array([[ 67,  23, 196, ...,  25,  25, 255],
       [ 41,  26, 229, ...,  84, 247, 136],
       [106, 102,  86, ...,  66, 175, 237],
       ...,
       [165, 152,  74, ..., 212, 182, 123],
       [111,  58,  99, ..., 228, 225,  14],
       [170, 244,  13, ...,  74, 106, 140]], dtype=uint8)

In [5]:
def selection_function_gen(n):
    return lambda v,g: lascar.tools.aes.sbox[v[n] ^ g] & 0x80 == 0

In [8]:
#[d["trace"][x] for x in range(1000) if (sbox[guess ^ d["input"][:,guess_idx][x]] & 0x1) == 0]
def selection_function(value, guess):
    return lascar.tools.aes.sbox[value[0] ^ guess] & 0x80 == 0


trace = lascar.TraceBatchContainer(data["trace"], data["input"])

engine = lascar.DpaEngine(
    selection_function=selection_function_gen(0),
    guess_range=range(256),
)

session = lascar.Session(
    trace,
    engine=engine,
    output_method=lascar.DictOutputMethod(engine),
)

asd = session.run(batch_size="auto")
asd.__dict__

2023-12-07 15:23:19,353 - lascar.session - INFO - Session Session: 1000 traces, 3 engines, batch_size=21451, leakage_shape=(1368,)
INFO:lascar.session:Session Session: 1000 traces, 3 engines, batch_size=21451, leakage_shape=(1368,)
Session |  0%||0 trc/1000 | (3 engines, batch_size=21451, leakage_shape=(1368,)) |ETA:  --:--:--


Session |100%||1000 trc/1000 | (3 engines, batch_size=21451, leakage_shape=(1368,)) |ETA:  00:00:00
Session |100%||1000 trc/1000 | (3 engines, batch_size=21451, leakage_shape=(1368,)) |Time:  0:00:01


{'logger': <Logger lascar.session (INFO)>,
 'container': <lascar.container.container.TraceBatchContainer at 0x7fb106424130>,
 'leakage_shape': (1368,),
 'value_shape': (16,),
 'name': 'Session',
 'engines': {'mean': <lascar.engine.engine.MeanEngine at 0x7fb1053ed120>,
  'var': <lascar.engine.engine.VarEngine at 0x7fb1053ec790>,
  'dpa': <lascar.engine.dpa_engine.DpaEngine at 0x7fb1053ecb50>},
 '_output_method': <lascar.output.pickle_output_method.DictOutputMethod at 0x7fb17b6b0730>,
 '_output_steps': [1000],
 '_progressbar': True,
 '_batch_size': 21451,
 '_thread_on_update': True}